In [ ]:
%pip install -q -U "google-genai>=1.43.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.1/256.1 kB 14.1 MB/s eta 0:00:00


## Gemini File Search Tool API: Getting started with file grounding for Gemini models

<a target="_blank" href="https://colab.research.google.com/drive/1qOH73lDHMEUk_J0Zo_rwk_6tW0L1Ix1I?usp=sharing"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

In [ ]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# !wget https://arxiv.org/pdf/1706.03762.pdf -O document.pdf

!wget https://storage.courtlistener.com/recap/gov.uscourts.cand.433688/gov.uscourts.cand.433688.340.1.pdf -O document.pdf

--2025-11-09 05:45:19--  https://storage.courtlistener.com/recap/gov.uscourts.cand.433688/gov.uscourts.cand.433688.340.1.pdf
Resolving storage.courtlistener.com (storage.courtlistener.com)... 18.65.39.29, 18.65.39.128, 18.65.39.110, ...
Connecting to storage.courtlistener.com (storage.courtlistener.com)|18.65.39.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 229009 (224K) [application/pdf]
Saving to: ‘document.pdf’

document.pdf        100%[===================>] 223.64K  --.-KB/s    in 0.01s   

2025-11-09 05:45:19 (22.2 MB/s) - ‘document.pdf’ saved [229009/229009]



In [ ]:
from google import genai
from google.genai import types
import time

In [ ]:
client = genai.Client()

# Create the file search store with an optional display name
file_search_store = client.file_search_stores.create(config={'display_name': 'sam-basic'})

In [ ]:
file_search_store

FileSearchStore(
  create_time=datetime.datetime(2025, 11, 9, 5, 46, 29, 539720, tzinfo=TzInfo(UTC)),
  display_name='sam-basic',
  name='fileSearchStores/sambasic-luhezy3p7ter',
  update_time=datetime.datetime(2025, 11, 9, 5, 46, 29, 539720, tzinfo=TzInfo(UTC))
)

In [ ]:
# Create a file search store (including optional display_name for easier reference)
# file_search_store = client.file_search_stores.create(config={'display_name': 'my-file_search-store-123'})

# List all your file search stores
for file_search_store in client.file_search_stores.list():
    print(file_search_store)

name='fileSearchStores/sambasic-luhezy3p7ter' display_name='sam-basic' create_time=datetime.datetime(2025, 11, 9, 5, 46, 29, 539720, tzinfo=TzInfo(UTC)) update_time=datetime.datetime(2025, 11, 9, 5, 46, 29, 539720, tzinfo=TzInfo(UTC)) active_documents_count=None pending_documents_count=None failed_documents_count=None size_bytes=None


In [ ]:
# Upload and import a file into the file search store, supply a file name which will be visible in citations
operation = client.file_search_stores.upload_to_file_search_store(
  file='/content/document.pdf',
  file_search_store_name=file_search_store.name,
  config={
      'display_name' : 'Illya Testimony 01',
  }
)

In [ ]:
client.operations.get(operation)

UploadToFileSearchStoreOperation(
  done=True,
  name='fileSearchStores/sambasic-luhezy3p7ter/upload/operations/illya-testimony-01-e6l53qvklyw5',
  response=UploadToFileSearchStoreResponse(
    document_name='fileSearchStores/sambasic-luhezy3p7ter/documents/illya-testimony-01-e6l53qvklyw5',
    parent='fileSearchStores/sambasic-luhezy3p7ter'
  )
)

In [ ]:
# Wait until import is complete
while not operation.done:
    time.sleep(5)
    print("Waiting")
    operation = client.operations.get(operation)

print("Done")

Done


In [ ]:
PROMPT = "Tell me about this document"

#  Ask a question about the file
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=f"""{PROMPT}\n (return your answer in markdown as sections and bullet points)\nANSWER:\n""",
    config=types.GenerateContentConfig(
        tools=[
            types.Tool(
                file_search=types.FileSearch(
                    file_search_store_names=[file_search_store.name]
                )
            )
        ]
    )
)

print(response.text)

This document is a highly confidential videotaped deposition transcript of Ilya Sutskever, taken on October 1, 2025, in San Francisco, California. The deposition is part of a legal case titled "MUSK v. ALTMAN" in the United States District Court, Northern District of California, Oakland Division, Case No. 4:24-cv-04722-YGR.

### Key Aspects of the Document:

*   **Deponent:** Ilya Sutskever.
*   **Date of Deposition:** Wednesday, October 1, 2025, commencing at 10:19 AM and concluding at 8:07 PM.
*   **Legal Case:** *Elon Musk, et al. v. Samuel Altman, et al.*
*   **Confidentiality:** The document is marked as "Highly Confidential."

### Content Regarding a Prepared Memo (Exhibit 19):

*   **Preparation:** Ilya Sutskever prepared a document referred to as "Exhibit 19" at the request of independent board members, most likely Adam D'Angelo.
*   **Purpose:** The memo was intended to "paint a picture from a large number of small pieces of evidence or items," utilizing screenshots largely ob

In [ ]:
print(response.candidates[0].grounding_metadata)

google_maps_widget_context_token=None grounding_chunks=[GroundingChunk(
  retrieved_context=GroundingChunkRetrievedContext(
    text="""--- PAGE 1 ---

EXHIBIT 4


--- PAGE 2 ---

In the Matter Of:

MUSK v

ALTMAN

ILYA SUTSKEVER

October 01, 2025

LEXITAS™


--- PAGE 3 ---

1

2

MUSK v
 ALTMAN

Highly Confidential

UNITED STATES DISTRICT COURT

NORTHERN OF CALIFORNIA

OAKLAND DIVISION

3

4

ELON MUSK, et al.,

)

5

Plaintiffs,

)

6

V.

)

7

SAMUEL ALTMAN, et al.,

)

)

8

Defendants.

)

9

10

11

12

13

14

15

16

17

18

19

20

21

22

23

24

25

Ilya Sutskever
 October 01, 2025

1

Case No. 4:24-cv-04722-YGR

** HIGHLY CONFIDENTIAL **

Videotaped Deposition of ILYA SUTSKEVER

San Francisco, California

Wednesday, October 1, 2025

Reported Stenographically by

Michael P. Hensley, RDR, CSR No. 14114

www.LexitasLegal.com/Premier

Lexitas

888-267-1200 Page 1


--- PAGE 4 ---

1

2

MUSK v
 ALTMAN

Highly Confidential

UNITED STATES DISTRICT COURT

NORTHERN OF CALIFORNIA



In [ ]:
for key in response.candidates[0].grounding_metadata.__dict__.keys():
    print(key)

google_maps_widget_context_token
grounding_chunks
grounding_supports
retrieval_metadata
retrieval_queries
search_entry_point
source_flagging_uris
web_search_queries


In [ ]:
response.candidates[0].grounding_metadata.grounding_supports[1], response.candidates[0].grounding_metadata.grounding_chunks[1]

(GroundingSupport(
   grounding_chunk_indices=[
     1,
   ],
   segment=Segment(
     end_index=1103,
     start_index=1048,
     text='He believed that termination was the appropriate action'
   )
 ),
 GroundingChunk(
   retrieved_context=GroundingChunkRetrievedContext(
     text="""this?
 
 A.
 
 I had discussions with the independent
 
 4
 
 13:53:56
 
 5
 
 Ilya Sutskever
  October 01, 2025
 
 130
 
 board members discussing the subject matter of these
  documents. And after having some discussions,
 
 6
 
 7
 
 either Adam or the three of them together, I don't
 
 8
 
 remember, have asked me to collect supporting
 
 9 screenshots.
 
 13:54:11 10
 
 Q.
 
 Okay.
 
 11
 
 And the three of them together were Adam
 
 12
 
 D'Angelo, Helen Toner, and Tasha McCauley?
 
 13
 
 A.
 
 Correct.
 
 14
 
 Q.
 
 All right.
 
 13:54:20 15
 
 And the document that you prepared, the
 
 16 very first page says:
 
 17
 
 18
 
 19
 
 13:54:31 20
 
 21
 
 A.
 
 [As Read] Sam exhibits a consistent
  

In [ ]:
PROMPT = "What did Ilya see?"

#  Ask a question about the file
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=f"""{PROMPT}\n (return your answer in markdown as sections and bullet points)\nANSWER:\n""",
    config=types.GenerateContentConfig(
        tools=[
            types.Tool(
                file_search=types.FileSearch(
                    file_search_store_names=[file_search_store.name]
                )
            )
        ]
    )
)

print(response.text)

### Deleting Your Stores

In [ ]:
# deleting the stores
# List all your file search stores
for file_search_store in client.file_search_stores.list():
    name = file_search_store.name
    print(name)
    # Get a specific file search store by name
    my_file_search_store = client.file_search_stores.get(name=file_search_store.name)

    # Delete a file search store
    client.file_search_stores.delete(name=name, config={'force': True})

fileSearchStores/sambasic-luhezy3p7ter
